# Setting up the environment

To modify Google Sheets spreadsheets using Python, the Google Sheets API was required. Through the four steps below:

- **Step 1**: Create a project on <a href="https://console.cloud.google.com/">*Google Cloud*</a>;
- **Step 2**: Select the created project, and activate the necessary APIs;
- **Step 3**: Configure the *OAuth* permission screen;
- **Step 4**: Authorize credentials for a computer application.

After this setup, the **credentials.json** file was generated, and then the Google client library for Python was installed. For that, the following code was executed at the VSCode terminal:
```
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
```

# Creating the app

After setting up the environment, Python commands finally could be executed. In the first place, the required libraries were imported:

In [1]:
import os.path
import math
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

Next, the API scopes, the spreadsheet, and its cell range to be used were set up:

In [2]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# The ID and range of the spreadsheet
SPREADSHEET_ID = "1y1beFCzEiD4xYjzVczTWXW4TDkemwNolMb-aPptoJKs"
RANGE_NAME = "engenharia_de_software!A4:F27"

# The missing spreadsheet data to update
DATA_TO_UPDATE = "engenharia_de_software!G4:H27"

Then, the following code was executed to update the blank spreadsheet values:

In [3]:
def school_status(school_absences, average_grade, total_classes):

  if school_absences > (total_classes * 0.25):
    situation = "Reprovado por Falta"
  else:
    if average_grade < 50:
      situation = "Reprovado por Nota"
    elif average_grade >= 50 and average_grade < 70:
      situation = "Exame Final"
    elif average_grade >= 70:
      situation = "Aprovado"
    
  if situation == "Exame Final":
    # The math.ceil method rounds a number up to the nearest integer
    minimum_grade = math.ceil(100 - average_grade)
    approval_grade = f"naf >= {minimum_grade}"
  else:
    approval_grade = 0
  
  return situation, approval_grade

def main():

  # The following lines let the user access the application

  # Start with empty creds
  creds = None

  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  # The following lines manipulate the data in Google Sheets
  try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()

    # Getting the number of classes in the semester
    data_total_classes = sheet.values().get(
      spreadsheetId=SPREADSHEET_ID,
      range="A2:H2").execute()
    total_classes = int(
      data_total_classes["values"][0][0].replace(
      "Total de aulas no semestre: ", ""
    ))

    # Getting the students information
    students_data = sheet.values().get(
      spreadsheetId=SPREADSHEET_ID,
      range=RANGE_NAME).execute()
    students_values = students_data['values']

    # Adding data
    data_to_add = []

    for student_info in students_values:
      absences = int(student_info[2])
      exam_1, exam_2, exam_3 = float(student_info[3]), float(student_info[4]), float(student_info[5])
      average_grade = (exam_1 + exam_2 + exam_3) / 3
      situation, approval_grade = school_status(absences, average_grade, total_classes)
      data_to_add.append([situation, approval_grade])

    # Updating the data
    sheet.values().update(
      spreadsheetId=SPREADSHEET_ID,
      range=DATA_TO_UPDATE, valueInputOption="USER_ENTERED",
      body={"values": data_to_add}).execute()

  except HttpError as err:
    print(err)

if __name__ == "__main__":
  main()
  print("Data updated!")

Data updated!
